In [0]:
# SETUP DO NOTEBOOK

%pip install openpyxl
%pip install yfinance
%pip freeze > requirements.txt

import pyspark
import pandas as pd
import yfinance as yf

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 45.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 116.9 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=a0c55cb882bc8706117de8811a42639978d3dc9d3ca9e6d6ad987012c81ee7e9
  Stored in directory: /home/spark-d5381db9-fea5-44d1-adfc-be/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b5

In [0]:
# INICIO DA BUSCA DOS DADOS

ticker = yf.Ticker("BZ=F")  # Brent
df = ticker.history(period="max")

display(df.head(10))

df.to_csv("../Bronze_data/brent_oil.csv")
"""
arquivo combustiveis-liquidos-estados.xlsx baixado de https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-de-distribuicao-de-combustiveis e upado para o databricks manualmente
^^^^
DADOS COLETADOS 09/12/2025
Aparentam ter as medições completas de 09/2020 até meados de 03/2025 a análise tratará os dados no período 09/2020 até 02/2025 para possibilitar um período de análise completo em ambas as bases
"""

# FINALIZAÇÃO DO PRIMEIRO NÍVEL DE DADOS - BRONZE (DADOS BRUTOS)

Open,High,Low,Close,Volume,Dividends,Stock Splits
75.8499984741211,76.52999877929688,75.44000244140625,75.73999786376953,2575,0.0,0.0
75.69999694824219,77.16999816894531,75.66999816894531,77.05000305175781,3513,0.0,0.0
77.0,77.05999755859375,74.86000061035156,75.3499984741211,3930,0.0,0.0
75.22000122070312,76.20999908447266,74.2699966430664,75.76000213623047,6180,0.0,0.0
75.38999938964844,76.0,74.52999877929688,74.75,4387,0.0,0.0
74.54000091552734,74.5999984741211,70.87000274658203,71.16999816894531,7201,0.0,0.0
70.80000305175781,72.01000213623047,70.55999755859375,71.80000305175781,6775,0.0,0.0
71.44000244140625,72.02999877929688,70.66000366210938,70.98999786376953,5768,0.0,0.0
70.91000366210938,71.08000183105469,69.2699966430664,70.20999908447266,6232,0.0,0.0
69.91000366210938,70.44999694824219,68.97000122070312,70.38999938964844,3947,0.0,0.0


'\narquivo combustiveis-liquidos-estados.xlsx baixado de https://www.gov.br/anp/pt-br/assuntos/precos-e-defesa-da-concorrencia/precos/precos-de-distribuicao-de-combustiveis e upado para o databricks manualmente\n^^^^\nDADOS COLETADOS 09/12/2025\nAparentam ter as medições completas de 09/2020 até meados de 03/2025 a análise tratará os dados no período 09/2020 até 02/2025 para possibilitar um período de análise completo em ambas as bases\n'

In [0]:
# INICIO DA TRANSFORMAÇÃO DE DADOS

# TRANSFORMAÇÃO DE DADOS DA BASE DE PETROLEO

df = pd.read_csv("../Bronze_data/brent_oil.csv")

df = df.drop(columns=["Dividends", "Stock Splits"])
df = df.rename(columns={"Date": "date", "Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"})

df["date"] = pd.to_datetime(df["date"], utc=True)
df = df[df['date'] < "2025-03-01"]
df = df[df['date'] > "2020-09-01"]

df = df[["date", "open", "high", "low", "close", "volume"]]
df = df.sort_values("date")
df = df.dropna()
df = df.astype({"open": "float64", "high": "float64", "low": "float64", "close": "float64", "volume": "int64"})
df = df.drop_duplicates()

df = (
    df.groupby(df["date"].dt.to_period("M"))
        .agg(
            avg_open=("open", "mean"),
            avg_close=("close", "mean"),
            avg_high=("high", "mean"),
            avg_low=("low", "mean"),
            highest=("high", "max"),
            lowest=("low", "min")
        )
        .reset_index()
)

df["date"] = df["date"].dt.strftime("%Y-%m")

df.to_csv("../Silver_data/brent_oil.csv")

display(df.head(10))

/home/spark-d5381db9-fea5-44d1-adfc-be/.ipykernel/2550/command-6355522900556503-4003098069:21: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df.groupby(df["date"].dt.to_period("M"))


date,avg_open,avg_close,avg_high,avg_low,highest,lowest
2020-09,42.04666646321615,41.86714299519857,42.647619156610396,41.16999980381557,46.220001220703125,39.27999877929688
2020-10,41.650000138716265,41.524091026999734,42.25681842457164,40.72545485063033,43.59999847412109,36.68000030517578
2020-11,43.29500026702881,43.78900012969971,44.394499969482425,42.68450031280518,49.06999969482422,35.72999954223633
2020-12,50.01681865345348,50.218181956898086,50.75227303938432,49.409545551646836,52.470001220703125,46.77000045776367
2021-01,55.07789491352282,55.35157916420385,55.98263148257607,54.49052670127467,57.41999816894531,50.58000183105469
2021-02,61.72421083952251,62.22789423089279,62.79052573756168,61.152631659256784,67.69999694824219,54.619998931884766
2021-03,65.70087001634681,65.70217381352964,66.98695638905401,64.31086946570355,71.36000061035156,60.2599983215332
2021-04,65.10666710989815,65.32857150123233,66.02857081095378,64.34285717918759,68.94999694824219,61.2400016784668
2021-05,68.16450004577636,68.25849990844726,69.02900047302246,67.13999938964844,70.26000213623047,64.55999755859375
2021-06,73.1199996254661,73.40818162397905,73.98318134654652,72.49363604458895,76.58999633789062,68.7699966430664


In [0]:
# TRANSFORMAÇÃO DE DADOS DA BASE DOS COMBUSTIVEIS

# Funcao para unificar diferentes sub-tipos de combustiveis
def categorize(prod):
    prod = prod.upper()
    if "GASOLINA" in prod:
        return "GASOLINE"
    if "DIESEL" in prod:
        return "DIESEL"
    if "ETANOL" in prod:
        return "ETHANOL"
    return None

# Ignora informacoes no inicio do arquivo
df = pd.read_excel("../Bronze_data/combustiveis-liquidos-estados.xlsx", skiprows=8)

df = df.drop(columns=["UNIDADE DE MEDIDA", "DESVIO PADRÃO"])
df = df.rename(columns={"MÊS": "date", "PRODUTO": "product", "REGIÃO": "region", "ESTADO": "state", "PRODUTO": "product", "PREÇO MÉDIO DE DISTRIBUIÇÃO": "price"})

df["price"] = pd.to_numeric(df["price"], errors="coerce")

df = df.dropna()
df = df.astype({"price": "float64"})
df = df.drop_duplicates()

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df[df['date'] < "2025-03-01"]
df = df.sort_values("date")

df['product'] = df['product'].apply(categorize)

df = (
    df
    .groupby(['date', 'product', 'region', 'state'], as_index=False)
    .agg(
        avg_price=('price', 'mean')
        )
    )

df.to_csv("../Silver_data/combustiveis.csv")

display(df.head(10))

# FINALIZADA A TRANSFORMAÇÃO DOS DADOS AO NÍVEL SILVER (PADRONIZAÇÃO)

date,product,region,state,avg_price
2020-09-01T00:00:00.000Z,DIESEL,CENTRO OESTE,DISTRITO FEDERAL,3.2376890400967815
2020-09-01T00:00:00.000Z,DIESEL,CENTRO OESTE,GOIAS,3.1620676493092272
2020-09-01T00:00:00.000Z,DIESEL,CENTRO OESTE,MATO GROSSO,3.296971282272546
2020-09-01T00:00:00.000Z,DIESEL,CENTRO OESTE,MATO GROSSO DO SUL,3.0930768137434743
2020-09-01T00:00:00.000Z,DIESEL,NORDESTE,ALAGOAS,3.22700850792022
2020-09-01T00:00:00.000Z,DIESEL,NORDESTE,BAHIA,3.0300035066521853
2020-09-01T00:00:00.000Z,DIESEL,NORDESTE,CEARA,3.1636475260003656
2020-09-01T00:00:00.000Z,DIESEL,NORDESTE,MARANHAO,3.0750825055990427
2020-09-01T00:00:00.000Z,DIESEL,NORDESTE,PARAIBA,3.0924569380938816
2020-09-01T00:00:00.000Z,DIESEL,NORDESTE,PERNAMBUCO,3.0146433402484996


In [0]:
# TRANSFORMAÇÃO FINAL DOS DADOS E CRIAÇÃO DA TABELA FATO - GOLD

df_oil = pd.read_csv("../Silver_data/brent_oil.csv")
df_gas = pd.read_csv("../Silver_data/combustiveis.csv", index_col=0)

df = pd.DataFrame()
df['date'] = df_oil['date']
df['oil_avg_price'] = (df_oil['avg_close'] + df_oil['avg_open'] + df_oil['avg_high'] + df_oil['avg_low'])/4
df['oil_pct_variation'] = df['oil_avg_price'].pct_change()

monthly_gas = (
    df_gas
    .groupby(['date', 'product'], as_index=False)
    .agg(
        avg_price=('avg_price', 'mean'),
    )
)

df['date'] = pd.to_datetime(df['date'], errors="coerce").dt.date
df_gas['date'] = pd.to_datetime(df_gas['date'], errors="coerce").dt.date

df['gasoline_avg_price'] = monthly_gas.loc[monthly_gas['product'] == 'GASOLINE', 'avg_price'].values
df['gasoline_pct_variation'] = df['gasoline_avg_price'].pct_change()

df['diesel_avg_price'] = monthly_gas.loc[monthly_gas['product'] == 'DIESEL', 'avg_price'].values
df['diesel_pct_variation'] = df['diesel_avg_price'].pct_change()

df['ethanol_avg_price'] = monthly_gas.loc[monthly_gas['product'] == 'ETHANOL', 'avg_price'].values
df['ethanol_pct_variation'] = df['ethanol_avg_price'].pct_change()

df['dif_pct_oil_gasoline'] = abs(df['oil_pct_variation']) - abs(df['gasoline_pct_variation'])
df['dif_pct_oil_diesel'] = abs(df['oil_pct_variation']) - abs(df['diesel_pct_variation'])
df['dif_pct_oil_ethanol'] = abs(df['oil_pct_variation']) - abs(df['ethanol_pct_variation'])

df = df.dropna()

df.to_csv("../Gold_data/tabela_fato.csv")

# Transformacao para DataFrame Spark, para possibilitar a criação da tabela no catalog do Databricks e sua utilizacao nos Dashboards
spark_df = spark.createDataFrame(df)
spark_df_gas = spark.createDataFrame(df_gas)
spark.sql("USE CATALOG `mvp-juj`")
spark.sql("USE SCHEMA `mvp-data`")
spark.sql("DROP TABLE IF EXISTS tabela_fato")
spark.sql("DROP TABLE IF EXISTS combustiveis")
spark_df.write.mode("overwrite").saveAsTable("tabela_fato")
spark_df_gas.write.mode("overwrite").saveAsTable("combustiveis")

display(df.head(10))

# FINALIZADA A CRIAÇÃO DA TABELA FATO - GOLD; DADOS PRONTOS PARA ANÁLISE

date,oil_avg_price,oil_pct_variation,gasoline_avg_price,gasoline_pct_variation,diesel_avg_price,diesel_pct_variation,ethanol_avg_price,ethanol_pct_variation,dif_pct_oil_gasoline,dif_pct_oil_diesel,dif_pct_oil_ethanol
2020-10-01,41.53909111022949,-0.009390392681743798,4.000581475001055,0.019991372340480318,3.161117152115135,0.006287070068495604,2.940048715987536,0.026836217559103037,-0.01060097965873652,0.003103322613248194,-0.01744582487735924
2020-11-01,43.54075016975403,0.04818735812521435,4.022281350632928,0.005424180401641943,3.2797494118533965,0.03752858689811078,3.030852934920966,0.030885276981857723,0.04276317772357241,0.01065877122710357,0.01730208114335663
2020-12-01,50.099204800345674,0.1506279658715559,4.142575636575075,0.02990697951132093,3.403301518154331,0.03767120312741046,3.059425726277037,0.009427310387402965,0.12072098636023498,0.11295676274414546,0.14120065548415295
2021-01-01,55.22565806539435,0.10232604061239114,4.244039399024149,0.02449291729358971,3.463267348709752,0.017619899452206544,3.1401698685211583,0.02639192759301845,0.07783312331880143,0.0847061411601846,0.0759341130193727
2021-02-01,61.97381561680844,0.12219243351384601,4.570447532959905,0.07690977939809152,3.6946167142567776,0.06680089702956815,3.333277615622712,0.06149595569251698,0.045282654115754495,0.05539153648427786,0.06069647782132903
2021-03-01,65.67521742115851,0.05972525279444274,5.065319441004941,0.10827646624892928,3.9390803859389827,0.06616753254508634,3.917253236762631,0.17519561479154566,-0.04855121345448654,-0.006442279750643598,-0.11547036199710292
2021-04-01,65.20166665031797,-0.007210494147339763,5.081333665428387,0.003161542842452736,3.8952258712100365,-0.011133186031310749,3.815672494631711,-0.025931625042160955,0.004048951304887027,-0.003922691883970986,-0.018721130894821192
2021-05-01,68.14799995422364,0.045188006001550374,5.194301802953071,0.022231985727148684,4.183299395728782,0.07395553789266063,4.168121187700547,0.09236869609870801,0.02295602027440169,-0.028767531891110254,-0.04718069009715764
2021-06-01,73.25124966014516,0.07488480526720487,5.267838528461761,0.014157191533784586,4.21236772752611,0.006948661582053539,4.323432784421537,0.03726177568427946,0.06072761373342028,0.06793614368515133,0.037623029582925405
2021-07-01,74.04238110496884,0.010800245026456157,5.378553341408294,0.021017123502998114,4.296123230718457,0.01988323636729028,4.3395595952456185,0.0037300940313425013,-0.010216878476541957,-0.009082991340834123,0.007070150995113655


In [0]:
# DEMAIS ANÁLISES NECESSÁRIAS

def corr_with_lag(df, col1, col2, lag):
    return df[col1].corr(df[col2].shift(-lag))

corr_gas_lag0 = corr_with_lag(df, 'oil_pct_variation', 'gasoline_pct_variation', 0)
corr_gas_lag1 = corr_with_lag(df, 'oil_pct_variation', 'gasoline_pct_variation', 1)
corr_gas_lag2 = corr_with_lag(df, 'oil_pct_variation', 'gasoline_pct_variation', 2)

corr_diesel_lag0 = corr_with_lag(df, 'oil_pct_variation', 'diesel_pct_variation', 0)
corr_diesel_lag1 = corr_with_lag(df, 'oil_pct_variation', 'diesel_pct_variation', 1)
corr_diesel_lag2 = corr_with_lag(df, 'oil_pct_variation', 'diesel_pct_variation', 2)

corr_ethanol_lag0 = corr_with_lag(df, 'oil_pct_variation', 'ethanol_pct_variation', 0)
corr_ethanol_lag1 = corr_with_lag(df, 'oil_pct_variation', 'ethanol_pct_variation', 1)
corr_ethanol_lag2 = corr_with_lag(df, 'oil_pct_variation', 'ethanol_pct_variation', 2)

corr_gas_etanol_lag0 = corr_with_lag(df, 'gasoline_pct_variation', 'ethanol_pct_variation', 0)
corr_gas_etanol_lag1 = corr_with_lag(df, 'gasoline_pct_variation', 'ethanol_pct_variation', 1)
corr_gas_etanol_lag2 = corr_with_lag(df, 'gasoline_pct_variation', 'ethanol_pct_variation', 2)

corr_gas_diesel_lag0 = corr_with_lag(df, 'gasoline_pct_variation', 'diesel_pct_variation', 0)
corr_gas_diesel_lag1 = corr_with_lag(df, 'gasoline_pct_variation', 'diesel_pct_variation', 1)
corr_gas_diesel_lag2 = corr_with_lag(df, 'gasoline_pct_variation', 'diesel_pct_variation', 2)

corr_diesel_etanol_lag0 = corr_with_lag(df, 'diesel_pct_variation', 'ethanol_pct_variation', 0)
corr_diesel_etanol_lag1 = corr_with_lag(df, 'diesel_pct_variation', 'ethanol_pct_variation', 1)
corr_diesel_etanol_lag2 = corr_with_lag(df, 'diesel_pct_variation', 'ethanol_pct_variation', 2)

results = pd.DataFrame({
    'correlation': ['Diesel x Oil', 'Gasoline x Oil', 'Ethanol x Oil', 'Gasoline x Ethanol', 'Gasoline x Diesel', 'Diesel x Ethanol'],
    'lag_0': [corr_diesel_lag0, corr_gas_lag0, corr_ethanol_lag0, corr_gas_etanol_lag0, corr_gas_diesel_lag0, corr_diesel_etanol_lag0],
    'lag_1': [corr_diesel_lag1, corr_gas_lag1, corr_ethanol_lag1, corr_gas_etanol_lag1, corr_gas_diesel_lag1, corr_diesel_etanol_lag1],
    'lag_2': [corr_diesel_lag2, corr_gas_lag2, corr_ethanol_lag2, corr_gas_etanol_lag2, corr_gas_diesel_lag2, corr_diesel_etanol_lag2]
})

results.display()
results.to_csv("../Gold_data/correlacoes.csv")

# Transformacao para DataFrame Spark, para possibilitar a criação da tabela no catalog do Databricks e sua utilizacao nos Dashboards
spark_df_corr = spark.createDataFrame(results)
spark.sql("USE CATALOG `mvp-juj`")
spark.sql("USE SCHEMA `mvp-data`")
spark.sql("DROP TABLE IF EXISTS correlacoes")
spark_df_corr.write.mode("overwrite").saveAsTable("correlacoes")

correlation,lag_0,lag_1,lag_2
Diesel x Oil,0.5973374033508755,0.44982028555458625,0.305980606761814
Gasoline x Oil,0.48109535769314105,0.32585795947246493,0.13221426204875208
Ethanol x Oil,0.3118492728099515,0.3803011688361468,0.23972505765246013
Gasoline x Ethanol,0.7822869491234313,0.5029657091925901,0.3233853863214497
Gasoline x Diesel,0.375431430824576,0.3042397573675275,0.21682896948954283
Diesel x Ethanol,0.34939788418330986,0.18872946795939938,-0.13772427635361303


In [0]:
# CRIACAO DE TABELAS COM METRICAS SOBRE OS COMBUSTIVEIS POR REGIAO

monthly_regions = (
    df_gas
    .groupby(['date', 'product', 'region'], as_index=False)
    .agg(
        avg_price=('avg_price', 'mean'),
    )
)

south = monthly_regions.loc[monthly_regions['region'] == 'SUL']
southeast = monthly_regions.loc[monthly_regions['region'] == 'SUDESTE']

regions = pd.DataFrame()
regions['date'] = monthly_regions['date'].unique()
regions['date'] = pd.to_datetime(regions['date'], errors="coerce").dt.date

regions['south_gas_avg_price'] = south.loc[south['product'] == 'GASOLINE', 'avg_price'].values
regions['south_gas_pct_variation'] = regions['south_gas_avg_price'].pct_change()
regions['south_diesel_avg_price'] = south.loc[south['product'] == 'DIESEL', 'avg_price'].values
regions['south_diesel_pct_variation'] = regions['south_diesel_avg_price'].pct_change()
regions['south_ethanol_avg_price'] = south.loc[south['product'] == 'ETHANOL', 'avg_price'].values
regions['south_ethanol_pct_variation'] = regions['south_ethanol_avg_price'].pct_change()

regions['southeast_gas_avg_price'] = southeast.loc[southeast['product'] == 'GASOLINE', 'avg_price'].values
regions['southeast_gas_pct_variation'] = regions['southeast_gas_avg_price'].pct_change()
regions['southeast_diesel_avg_price'] = southeast.loc[southeast['product'] == 'DIESEL', 'avg_price'].values
regions['southeast_diesel_pct_variation'] = regions['southeast_diesel_avg_price'].pct_change()
regions['southeast_ethanol_avg_price'] = southeast.loc[southeast['product'] == 'ETHANOL', 'avg_price'].values
regions['southeast_ethanol_pct_variation'] = regions['southeast_ethanol_avg_price'].pct_change()

regions = regions.dropna()

display(regions.head(10))

# Transformacao para DataFrame Spark, para possibilitar a criação da tabela no catalog do Databricks e sua utilizacao nos Dashboards
regions.to_csv('../Gold_data/regions.csv')
spark_regions = spark.createDataFrame(regions)
spark.sql("USE CATALOG `mvp-juj`")
spark.sql("USE SCHEMA `mvp-data`")
spark.sql("DROP TABLE IF EXISTS regions")
spark_regions.write.mode('overwrite').saveAsTable('regions')

date,south_gas_avg_price,south_gas_pct_variation,south_diesel_avg_price,south_diesel_pct_variation,south_ethanol_avg_price,south_ethanol_pct_variation,southeast_gas_avg_price,southeast_gas_pct_variation,southeast_diesel_avg_price,southeast_diesel_pct_variation,southeast_ethanol_avg_price,southeast_ethanol_pct_variation
2020-10-01,3.9118932055833713,0.023773521308620094,2.9618748631162153,0.004314365307309664,2.933379881636634,0.04940025901003908,4.062534376809927,-0.006561514949613545,3.0636688709877804,0.007006407475695697,2.8819348449604942,0.07210126106274228
2020-11-01,3.8997352739400224,-0.0031079405813011496,3.067317073399227,0.03559981942386159,3.090858176265945,0.05368493034780353,4.181941010492186,0.02939215342124002,3.176839216838355,0.03693948354610743,3.0209774201846,0.0482462590947339
2020-12-01,4.198466977768613,0.0766030724764486,3.153697417042029,0.02816153060664006,3.1007219145331297,0.0031912620070784214,4.282224515012833,0.023980133691279493,3.2922435919933015,0.036326791278344484,3.026432440115861,0.0018057135729692142
2021-01-01,4.116077916534165,-0.01962360587107359,3.2407826163792595,0.02761368255135621,3.135404560852534,0.011185345630914556,4.3005334020061055,0.004275555130069497,3.3516657323391676,0.018049132357757625,3.0802117457837968,0.017769868230026242
2021-02-01,4.659214906402516,0.13195498260287697,3.4839505553652237,0.07503370875817694,3.304424673519365,0.05390695503130649,4.607541010348841,0.07138826272097387,3.5881918958010703,0.07056973527513088,3.33261014947969,0.08194190027402404
2021-03-01,4.942763650935164,0.06085762306070186,3.722846448938148,0.06857040298836292,3.871780835946312,0.17169589822203024,5.2603934540827035,0.14169216123470463,3.808307970415698,0.06134456601170335,3.8850299176045624,0.16576189333490454
2021-04-01,4.919025462255448,-0.0048026145606263615,3.674085158933947,-0.013097851515769365,3.6638221000404108,-0.05371139140293657,5.283071703406494,0.004311131766425058,3.7780415650915384,-0.007947467893689275,3.5598768850394786,-0.08369382976735662
2021-05-01,5.081638403463874,0.03305795882867124,3.9796694874570306,0.08317290299600733,4.118719089690045,0.12415913688730051,5.451578574230955,0.03189562441785698,4.074805908857745,0.07854978264618873,4.103040087909196,0.1525792100149257
2021-06-01,5.148488354159995,0.013155196294674099,3.981693487442458,5.085849445052304E-4,4.199882209791379,0.019705913011766274,5.478073949279394,0.004860128986066581,4.104987958169636,0.007406990660900226,4.166870955845627,0.015556969117734631
2021-07-01,5.218600041110027,0.013617916974286493,4.0596803411563895,0.01958635288223176,4.197837625750734,-4.8681937695260746E-4,5.611360356780808,0.024330888691078467,4.19026097511497,0.020773024869811474,4.140303555412917,-0.006375863499069823


In [0]:
# CRIACAO DE TABELAS COM METRICAS SOBRE OS COMBUSTIVEIS POR ESTADO

south = df_gas.loc[df_gas['region'] == 'SUL']
southeast = df_gas.loc[df_gas['region'] == 'SUDESTE']

PR = south.loc[south['state'] == 'PARANA']
SC = south.loc[south['state'] == 'SANTA CATARINA']
RS = south.loc[south['state'] == 'RIO GRANDE DO SUL']
SP = southeast.loc[southeast['state'] == 'SAO PAULO']
RJ = southeast.loc[southeast['state'] == 'RIO DE JANEIRO']
MG = southeast.loc[southeast['state'] == 'MINAS GERAIS']
ES = southeast.loc[southeast['state'] == 'ESPIRITO SANTO']

south = pd.DataFrame()
southeast = pd.DataFrame()

south['date'] = df_gas['date'].unique()
south['date'] = pd.to_datetime(south['date'], errors="coerce").dt.date
southeast['date'] = df_gas['date'].unique()
southeast['date'] = pd.to_datetime(southeast['date'], errors="coerce").dt.date

south['PR_gas_avg_price'] = PR.loc[PR['product'] == 'GASOLINE', 'avg_price'].values
south['PR_gas_pct_variation'] = south['PR_gas_avg_price'].pct_change()
south['PR_diesel_avg_price'] = PR.loc[PR['product'] == 'DIESEL', 'avg_price'].values
south['PR_diesel_pct_variation'] = south['PR_diesel_avg_price'].pct_change()
south['PR_ethanol_avg_price'] = PR.loc[PR['product'] == 'ETHANOL', 'avg_price'].values
south['PR_ethanol_pct_variation'] = south['PR_ethanol_avg_price'].pct_change()
south['SC_gas_avg_price'] = SC.loc[SC['product'] == 'GASOLINE', 'avg_price'].values
south['SC_gas_pct_variation'] = south['SC_gas_avg_price'].pct_change()
south['SC_diesel_avg_price'] = SC.loc[SC['product'] == 'DIESEL', 'avg_price'].values
south['SC_diesel_pct_variation'] = south['SC_diesel_avg_price'].pct_change()
south['SC_ethanol_avg_price'] = SC.loc[SC['product'] == 'ETHANOL', 'avg_price'].values
south['SC_ethanol_pct_variation'] = south['SC_ethanol_avg_price'].pct_change()
south['RS_gas_avg_price'] = RS.loc[RS['product'] == 'GASOLINE', 'avg_price'].values
south['RS_gas_pct_variation'] = south['RS_gas_avg_price'].pct_change()
south['RS_diesel_avg_price'] = RS.loc[RS['product'] == 'DIESEL', 'avg_price'].values
south['RS_diesel_pct_variation'] = south['RS_diesel_avg_price'].pct_change()
south['RS_ethanol_avg_price'] = RS.loc[RS['product'] == 'ETHANOL', 'avg_price'].values
south['RS_ethanol_pct_variation'] = south['RS_ethanol_avg_price'].pct_change()

southeast['SP_gas_avg_price'] = SP.loc[SP['product'] == 'GASOLINE', 'avg_price'].values
southeast['SP_gas_pct_variation'] = southeast['SP_gas_avg_price'].pct_change()
southeast['SP_diesel_avg_price'] = SP.loc[SP['product'] == 'DIESEL', 'avg_price'].values
southeast['SP_diesel_pct_variation'] = southeast['SP_diesel_avg_price'].pct_change()
southeast['SP_ethanol_avg_price'] = SP.loc[SP['product'] == 'ETHANOL', 'avg_price'].values
southeast['SP_ethanol_pct_variation'] = southeast['SP_ethanol_avg_price'].pct_change()
southeast['RJ_gas_avg_price'] = RJ.loc[RJ['product'] == 'GASOLINE', 'avg_price'].values
southeast['RJ_gas_pct_variation'] = southeast['RJ_gas_avg_price'].pct_change()
southeast['RJ_diesel_avg_price'] = RJ.loc[RJ['product'] == 'DIESEL', 'avg_price'].values
southeast['RJ_diesel_pct_variation'] = southeast['RJ_diesel_avg_price'].pct_change()
southeast['RJ_ethanol_avg_price'] = RJ.loc[RJ['product'] == 'ETHANOL', 'avg_price'].values
southeast['RJ_ethanol_pct_variation'] = southeast['RJ_ethanol_avg_price'].pct_change()
southeast['MG_gas_avg_price'] = MG.loc[MG['product'] == 'GASOLINE', 'avg_price'].values
southeast['MG_gas_pct_variation'] = southeast['MG_gas_avg_price'].pct_change()
southeast['MG_diesel_avg_price'] = MG.loc[MG['product'] == 'DIESEL', 'avg_price'].values
southeast['MG_diesel_pct_variation'] = southeast['MG_diesel_avg_price'].pct_change()
southeast['MG_ethanol_avg_price'] = MG.loc[MG['product'] == 'ETHANOL', 'avg_price'].values
southeast['MG_ethanol_pct_variation'] = southeast['MG_ethanol_avg_price'].pct_change()
southeast['ES_gas_avg_price'] = ES.loc[ES['product'] == 'GASOLINE', 'avg_price'].values
southeast['ES_gas_pct_variation'] = southeast['ES_gas_avg_price'].pct_change()
southeast['ES_diesel_avg_price'] = ES.loc[ES['product'] == 'DIESEL', 'avg_price'].values
southeast['ES_diesel_pct_variation'] = southeast['ES_diesel_avg_price'].pct_change()
southeast['ES_ethanol_avg_price'] = ES.loc[ES['product'] == 'ETHANOL', 'avg_price'].values
southeast['ES_ethanol_pct_variation'] = southeast['ES_ethanol_avg_price'].pct_change()

south = south.dropna()
southeast = southeast.dropna()

south.to_csv('../Gold_data/south.csv')
southeast.to_csv('../Gold_data/southeast.csv')

# Transformacao para DataFrame Spark, para possibilitar a criação da tabela no catalog do Databricks e sua utilizacao nos Dashboards
spark_south = spark.createDataFrame(south)
spark_southeast = spark.createDataFrame(southeast)
spark.sql("USE CATALOG `mvp-juj`")
spark.sql("USE SCHEMA `mvp-data`")
spark.sql("DROP TABLE IF EXISTS south")
spark.sql("DROP TABLE IF EXISTS southeast")
spark_south.write.mode("overwrite").saveAsTable("south")
spark_southeast.write.mode("overwrite").saveAsTable("southeast")

display(south.head(10))

date,PR_gas_avg_price,PR_gas_pct_variation,PR_diesel_avg_price,PR_diesel_pct_variation,PR_ethanol_avg_price,PR_ethanol_pct_variation,SC_gas_avg_price,SC_gas_pct_variation,SC_diesel_avg_price,SC_diesel_pct_variation,SC_ethanol_avg_price,SC_ethanol_pct_variation,RS_gas_avg_price,RS_gas_pct_variation,RS_diesel_avg_price,RS_diesel_pct_variation,RS_ethanol_avg_price,RS_ethanol_pct_variation
2020-10-01,3.857937905910916,0.023520973426703895,2.910299911642938,0.008830238742777219,2.46613380806252,0.05539608406795127,3.8025776704683913,0.023562110168620087,2.9867142418617507,0.0032720055326653785,2.9619956970987857,0.05702874619837539,4.075164040370806,0.02421016296205325,2.988610435843957,9.90325199725417E-4,3.372010139748595,0.038501916625010235
2020-11-01,3.847650633924365,-0.00266652088173569,3.010700771984411,0.03449845836843468,2.5866479811398597,0.04886765376774904,3.784885598841196,-0.004652652269168889,3.0824335234834037,0.03204835610988588,3.14935748359809,0.06325525276178534,4.066669589054507,-0.002084444020448939,3.108816924729866,0.04022153153325392,3.536569064059885,0.048801432229251596
2020-12-01,3.936663929043402,0.023134453615464956,3.12760601198854,0.03882990999702529,2.6206553893341504,0.013147288862748407,4.543153027521959,0.20034091094138207,3.178110942689627,0.031039572622510336,3.1297252748847693,-0.006233718723760551,4.115583976740479,0.012028119475854693,3.1553752964479207,0.014976234640159847,3.551785079380469,0.004302479336602216
2021-01-01,4.057214813288974,0.03062259985064708,3.185798467734484,0.0186060697936008,2.6525098397199605,0.012155146577247367,3.994271594439048,-0.12081508805841301,3.2654367069952164,0.027477254847398713,3.165020857362334,0.011277533769753711,4.296747341874472,0.044018872208136495,3.2711126744080774,0.03667943337530888,3.588682985475309,0.010388552592623501
2021-02-01,4.361699891519876,0.07504780797743171,3.4416799194673198,0.08031940950577465,2.8067156285723875,0.05813580275679864,4.988036643387706,0.24879756557671473,3.5054857854804307,0.073512090426062,3.3611456592191087,0.06196635368154291,4.627908184299966,0.07707244947780056,3.5046859611479206,0.07140484293532001,3.7454127327666,0.04367333306553767
2021-03-01,4.89091554049088,0.12133243050488596,3.688293162525534,0.071654903660066,3.333428421715611,0.18766161693806205,4.781890877269095,-0.04132803763418291,3.73997762217473,0.06689282200645463,3.8927162828468433,0.15815161778833287,5.155484535045518,0.11399888021446447,3.74026856211418,0.06721931824359428,4.38919780327648,0.1718862823522096
2021-04-01,4.839845324765074,-0.010441851899303245,3.634432351595748,-0.014603180538096172,3.1183826204028167,-0.06451190009417296,4.703447055422545,-0.016404352140161893,3.693581068402632,-0.01240556988817465,3.692491595065973,-0.05143572591281709,5.213784006578724,0.0113082429278768,3.6942420568034624,-0.012305668576028994,4.180592084652441,-0.04752707168228265
2021-05-01,5.010376971267833,0.03523493728821525,3.939985313601278,0.08407171531790181,3.526647710775946,0.13092206443877363,4.896918876826515,0.041134048948402224,4.004615090220174,0.08420933940731268,4.178398849861871,0.13159332723876283,5.337619362297273,0.023751531625072042,3.99440805854964,0.08125239145967167,4.651110708432316,0.11254832192483377
2021-06-01,5.082979557079582,0.014490443778600204,3.92631516075966,-0.0034695948724546843,3.5856844112691486,0.016740175184726036,4.9547070465369085,0.011800924451467187,3.99732149545685,-0.0018212973279593259,4.255039948540746,0.018342217062741417,5.407778458863493,0.013144267472835258,4.021443806110863,0.006768399013054038,4.75892226956424,0.02317974520289634
2021-07-01,5.147296459055486,0.012653385923286464,3.9986367823328814,0.01841971889980143,3.582282637366366,-9.487097894314012E-4,5.006512869548738,0.010455880140893381,4.073763212855954,0.019123234767577157,4.2562969819906105,2.9542224398970696E-4,5.501990794725859,0.0174216337779054,4.106641028280332,0.021185729871447068,4.754933257895224,-8.382174456869063E-4
